<a target="_blank"  href="https://colab.research.google.com/github/Redislabs-Solution-Architects/Redis-Workshops/blob/main/00-Setup/00-Setup_Redis_Workshops.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Redis Workshop

![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

This notebook, as most of the notebooks in this series are designed to run either with build-in RedisStack or Redis Cloud.

To provision free forever instance of Redis Cloud:
- Head to https://redis.io/try-free/
- Register with email/gmail
- Create an **Essentials** subscription with 30MB free tier (no credit card required)
- Create RedisStack Database

In [1]:
# Install the requirements
!pip install -q redis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.3/261.3 kB 4.7 MB/s eta 0:00:00


### Install Redis Stack locally

This installs the Redis Stack database for local usage if you cannot or do not want to create a Redis Cloud database, and it installs the `redis-cli` that will be used for checking database connectivity, etc.

In [2]:
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes


deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb jammy main
Starting redis-stack-server, database path /var/lib/redis-stack


### Setup Redis connection
***Note - Replace Redis connection values if using a Redis Cloud instance!***

In [3]:
import os

REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = os.getenv("REDIS_PORT", "6379")
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")
# Replace values above with your own if using Redis Cloud instance
#REDIS_HOST="redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
#REDIS_PORT=18374
#REDIS_PASSWORD="1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# Shortcut for redis-cli $REDIS_CONN command
# If SSL is enabled on the endpoint add --tls
if REDIS_PASSWORD!="":
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT} -a {REDIS_PASSWORD} --no-auth-warning"
else:
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT}"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"


### Basic Test with Redis CLI

In [4]:
!redis-cli $REDIS_CONN PING
!redis-cli $REDIS_CONN SET hello world
!redis-cli $REDIS_CONN GET hello

PONG
OK
"world"


### Basic Test with Python client

In [5]:
import redis

r = redis.Redis(
  host=REDIS_HOST,
  port=REDIS_PORT,
  password=REDIS_PASSWORD
)

r.get("hello")

b'world'

### TTL - Time To Live

It's extremely important for caching systems to control for how long values will be stored. Redis allows you to control expiry with the EXPIRE command and with flags like EX, EXAT etc. For more information see https://redis.io/commands/expire/ and https://redis.io/commands/set/

In [6]:
import time

r.set(name="ttl_key", value="this key will expire soon", ex=10)

for i in range(15):
  print(r.get(name="ttl_key"))
  time.sleep(1)

b'this key will expire soon'
b'this key will expire soon'
b'this key will expire soon'
b'this key will expire soon'
b'this key will expire soon'
b'this key will expire soon'
b'this key will expire soon'
b'this key will expire soon'
b'this key will expire soon'
b'this key will expire soon'
None
None
None
None
None


#### Cleanup
(Optional) Delete all keys and indexes from the database.

In [7]:
###-- FLUSHDB will wipe out the entire database!!! Use with caution --###
# !redis-cli $REDIS_CONN flushdb